# Part 3: MongoDB

In [1]:
HOST = "10.128.0.4"
PORT = "27017"
DB = "final_project"

In [2]:
from pymongo import MongoClient

def connect():
    url = "mongodb://{}:{}".format(HOST, PORT)
    connection = MongoClient(url)
    print(connection)
    return connection[DB]

### 1. Test Connection

In [3]:
from pymongo import MongoClient

connection = connect()

print(connection)

MongoClient(host=['10.128.0.4:27017'], document_class=dict, tz_aware=False, connect=True)
Database(MongoClient(host=['10.128.0.4:27017'], document_class=dict, tz_aware=False, connect=True), 'final_project')


### 2. Write records into table

In [7]:
from pymongo import MongoClient
import random
from faker import Faker
from faker_airtravel import AirTravelProvider
from datetime import datetime, timedelta

connection = connect()

fake = Faker()
fake.add_provider(AirTravelProvider)

curr_codes = ['KMF', 'MRO', 'KYD', 'TZS', 'PLN', 'RWF', 'CHF', 'PEN', 'AOA', 'XCD', 'LTL', 'SBD', 'USD', 'UAH', 'TRY', 
              'EUR', 'AWG', 'GGP', 'NOK', 'PHP', 'KES', 'SVC', 'GMD', 'CZK', 'MWK', 'STD', 'CLP', 'ALL', 'UGX', 'LAK']

records = []
for i in range(100):
    flight = fake.flight()
    while (flight["airline"] != "American Airlines" or flight["stops"] != "non-stop") and random.random() < 0.9:
        flight = fake.flight()
    dep_date = fake.date_time_between(start_date = datetime(2019, 1, 1))
    while dep_date < datetime(2023, 12, 1) and random.random() < 0.99:
        dep_date = fake.date_time_between(start_date = datetime(2019, 1, 1))
    duration = timedelta(random.random() * 5/3 + 1/12)
    record = {
        "tck_id": i,
        "cust_id": random.randint(0, 49),
        "airline": flight["airline"],
        "flight_nm": flight["origin"]["icao"] + str(random.randint(100, 1000)),
        "dep_airport": flight["origin"]["airport"],
        "arr_airport": flight["destination"]["airport"],
        "dep_date": datetime.combine(dep_date.date(), datetime.min.time()),
        "dep_time": datetime.combine(datetime.min.date(), dep_date.time()),
        "arr_date": datetime.combine((dep_date + duration).date(), datetime.min.time()),
        "arr_time": datetime.combine(datetime.min.date(), (dep_date + duration).time()),
        "stops": flight["stops"] if flight["stops"] != "non-stop" else 0,
        "tik_amt": round(random.random() * 5000, 2),
        "curr_code": random.choice(curr_codes)
    }
    print(record["dep_date"], record["dep_time"],record["arr_date"],record["arr_time"])
    records.append(record)

try:
    connection = connect()
    collection = connection["ticketing"]
    collection.delete_many({"tck_id": {"$gte": 0}})
    collection.insert_many(records)
    print("100 documents written into ticketing collection")
except Exception as e:
    print("Error occurred while writing documents:", e)

MongoClient(host=['10.128.0.4:27017'], document_class=dict, tz_aware=False, connect=True)
2020-01-22 00:00:00 0001-01-01 23:03:44.531024 2020-01-24 00:00:00 0001-01-01 05:00:36.688270
2023-08-16 00:00:00 0001-01-01 08:37:55.120725 2023-08-17 00:00:00 0001-01-01 07:44:54.094241
2023-04-22 00:00:00 0001-01-01 00:52:33.402135 2023-04-22 00:00:00 0001-01-01 11:57:45.571537
2023-04-09 00:00:00 0001-01-01 10:07:00.751718 2023-04-10 00:00:00 0001-01-01 14:49:42.949734
2023-12-05 00:00:00 0001-01-01 06:29:04.560335 2023-12-06 00:00:00 0001-01-01 17:41:15.079787
2021-07-26 00:00:00 0001-01-01 12:06:45.475427 2021-07-26 00:00:00 0001-01-01 17:07:10.935715
2023-12-06 00:00:00 0001-01-01 02:34:35.803070 2023-12-07 00:00:00 0001-01-01 11:54:04.784150
2021-08-12 00:00:00 0001-01-01 01:10:37.495546 2021-08-13 00:00:00 0001-01-01 03:33:17.057730
2020-03-01 00:00:00 0001-01-01 19:41:58.323406 2020-03-02 00:00:00 0001-01-01 18:00:27.315830
2021-05-12 00:00:00 0001-01-01 01:53:55.002920 2021-05-13 00:00: